# 판례/사례 파싱 테스트 노트북
## 주택임대차 RAG - 대법원판례, 하급심판례, 상담사례 파싱

### 1. 필요 패키지 임포트

In [1]:
from pathlib import Path
import pandas as pd
import re
import uuid
from datetime import datetime
from collections import Counter
import PyPDF2

print("✅ 패키지 임포트 완료")

✅ 패키지 임포트 완료


### 2. 카테고리 정의

In [2]:
def get_law_category():
    """주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전"""
    return {
        '보증금_대항력': {
            '보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3,
            '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2,
            '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2,
            '반환보증': 2, 'HUG': 1, '보증보험': 2
        },
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '실거주': 3, '존속기간': 2, '2년': 1,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1
        },
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2,
            '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3
        },
        '임대료_증감': {
            '차임': 3, '월세': 2, '증액': 3, '감액': 2, '인상': 2,
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1
        },
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2,
            '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3,
            '비용상환': 2, '보존행위': 2, '관리비': 1
        },
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2,
            '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3,
            '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2
        },
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1
        },
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2,
            '내용증명': 2, '손해배상': 3, '지연이자': 2
        },
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2,
            '공동상속': 2, '반환청구권': 2
        }
    }

print("✅ 카테고리 정의 완료")
print(f"총 {len(get_law_category())}개 카테고리")

✅ 카테고리 정의 완료
총 9개 카테고리


### 3. 카테고리 분류 함수

In [3]:
def categorize_content(content, top_k=None):
    """내용 카테고리 분류"""
    category_keywords = get_law_category()
    category_scores = {}
    
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            count = content.count(keyword)
            score += count * weight
        if score > 0:
            category_scores[category] = score
    
    sorted_categories = sorted(category_scores.items(), key=lambda x: x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    
    if not all_categories:
        all_categories = ["기타"]
    
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories

print("✅ 카테고리 분류 함수 정의 완료")

✅ 카테고리 분류 함수 정의 완료


### 4. 경로 설정

In [4]:
BASE_DIR = Path.cwd().parents[1]

DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
CSV_DIR = PROCESSED_DIR / "csv"
LOG_DIR = PROCESSED_DIR / "log"

CASE_RAW_DIR = RAW_DIR / "case"
CASE_CSV_PATH = CSV_DIR / "case.csv"

# 디렉토리 생성
CSV_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)
CASE_RAW_DIR.mkdir(parents=True, exist_ok=True)

print("📁 경로 설정 완료")
print(f"Raw 디렉토리: {CASE_RAW_DIR}")
print(f"CSV 저장 경로: {CASE_CSV_PATH}")

📁 경로 설정 완료
Raw 디렉토리: c:\project-AI2\실프로젝트\data\raw\case
CSV 저장 경로: c:\project-AI2\실프로젝트\data\processed\csv\case.csv


### 5. PDF 파싱 함수 정의

In [5]:
def extract_text_from_pdf(pdf_path):
    """PDF에서 텍스트 추출"""
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        print(f"❌ PDF 읽기 오류: {e}")
        return ""

print("✅ PDF 텍스트 추출 함수 정의 완료")

✅ PDF 텍스트 추출 함수 정의 완료


In [6]:
def extract_case_info(text, filename):
    """PDF 텍스트에서 사건번호, 제목, 날짜 등 추출"""
    case_no = "미상"
    ruling_date = "미상"
    src_title = filename
    
    # 사건번호 패턴
    case_pattern = re.search(r'(\d{4}[다가나마자]합?\d+)', text[:500])
    if case_pattern:
        case_no = case_pattern.group(1)
    
    # 날짜 패턴
    date_pattern = re.search(r'(\d{4})[\.\-년]\s*(\d{1,2})[\.\-월]\s*(\d{1,2})', text[:500])
    if date_pattern:
        year, month, day = date_pattern.groups()
        ruling_date = f"{year}{month.zfill(2)}{day.zfill(2)}"
    
    return case_no, ruling_date, src_title

def extract_related_articles(text):
    """텍스트에서 참조 법령 조항 추출"""
    articles = []
    
    law_pattern = re.findall(r'주택임대차보호법\s*제\s*(\d+)조', text)
    for art in law_pattern:
        articles.append(f"주임법 제{art}조")
    
    civil_pattern = re.findall(r'민법\s*제\s*(\d+)조', text)
    for art in civil_pattern:
        articles.append(f"민법 제{art}조")
    
    unique_articles = list(set(articles))
    return ", ".join(unique_articles) if unique_articles else "미상"

print("✅ 사건정보 추출 함수 정의 완료")

✅ 사건정보 추출 함수 정의 완료


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_text(text, chunk_size=1500, overlap=200):
    """RecursiveCharacterTextSplitter를 사용한 텍스트 청킹"""
    
    # 빈 텍스트 체크
    if not text or len(text.strip()) < 10:
        return []
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    
    chunks = text_splitter.split_text(text)
    
    # 빈 청크 필터링
    chunks = [chunk.strip() for chunk in chunks if chunk.strip() and len(chunk.strip()) >= 10]
    
    return chunks

print("✅ 텍스트 청킹 함수 정의 완료")

✅ 텍스트 청킹 함수 정의 완료


In [9]:
def parse_case_pdf(file_path, priority=7, chunk_size=1000):
    """판례/사례 PDF 파일을 파싱하여 청크 단위로 분리"""
    print(f"   📄 PDF 텍스트 추출 중...")
    full_text = extract_text_from_pdf(file_path)
    
    if not full_text:
        print(f"   ⚠️ 텍스트 추출 실패")
        return pd.DataFrame()
    
    filename = file_path.stem
    case_no, ruling_date, src_title = extract_case_info(full_text, filename)
    related_article = extract_related_articles(full_text)
    
    # 우선순위 자동 판단
    if "대법원" in full_text[:500] or "대법원" in filename:
        priority = 5
        src_title = f"대법원 판례 - {case_no}"
    elif "지방법원" in full_text[:500] or "지법" in filename:
        priority = 6
        src_title = f"하급심 판례 - {case_no}"
    elif "상담" in filename or "사례" in filename:
        priority = 7
        src_title = f"상담사례 - {filename}"
    
    chunks = chunk_text(full_text, chunk_size=chunk_size)
    
    rows = []
    for idx, chunk in enumerate(chunks):
        categories = categorize_content(chunk, top_k=3)
        
        rows.append({
            "id": str(uuid.uuid4()),
            "chunk_id": f"chunk_{idx:03d}",
            "text": chunk,
            "src_title": src_title,
            "case_no": case_no,
            "ruling_date": ruling_date,
            "related_article": related_article,
            "category": ", ".join(categories),
            "priority": priority,
            "source": str(file_path),
            "parsed_at": datetime.now().isoformat()
        })
    
    print(f"   ✅ {len(chunks)}개 청크 생성")
    return pd.DataFrame(rows)

print("✅ 판례 파싱 함수 정의 완료")

✅ 판례 파싱 함수 정의 완료


### 6. 파일 검색 및 파싱 실행

In [10]:
pdf_files = list(CASE_RAW_DIR.glob("*.pdf"))

print("=" * 60)
print("판례/사례 PDF 파일 파싱 시작")
print("=" * 60)

if not pdf_files:
    print(f"\n⚠️  판례/사례 파일이 없습니다.")
    print(f"📁 다음 경로에 PDF 파일을 추가해주세요:")
    print(f"   {CASE_RAW_DIR}")
else:
    print(f"\n📂 발견된 PDF 파일: {len(pdf_files)}개")
    for f in pdf_files:
        print(f"   - {f.name}")

판례/사례 PDF 파일 파싱 시작

📂 발견된 PDF 파일: 2개
   - 2025전세피해지원사례집.pdf
   - 전세피해법률상담사례집_250102.pdf


In [11]:
# 파싱 실행
all_dfs = []

for pdf_file in pdf_files:
    print(f"\n🔄 파싱 중: {pdf_file.name}")
    
    df = parse_case_pdf(
        file_path=pdf_file,
        priority=7,
        chunk_size=1000
    )
    
    if not df.empty:
        all_dfs.append(df)
    else:
        print(f"   ⚠️ 파싱 실패")

print("\n" + "=" * 60)
print("파싱 완료")
print("=" * 60)


🔄 파싱 중: 2025전세피해지원사례집.pdf
   📄 PDF 텍스트 추출 중...
   ✅ 50개 청크 생성

🔄 파싱 중: 전세피해법률상담사례집_250102.pdf
   📄 PDF 텍스트 추출 중...
   ✅ 110개 청크 생성

파싱 완료


### 7. 결과 저장 및 통계

In [12]:
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv(CASE_CSV_PATH, index=False, encoding="utf-8-sig")
    
    print(f"✅ case.csv 생성 완료!")
    print(f"📊 총 {len(final_df)} 청크 저장")
    print(f"📁 저장 경로: {CASE_CSV_PATH}")
else:
    print("❌ 파싱된 데이터가 없습니다.")

✅ case.csv 생성 완료!
📊 총 160 청크 저장
📁 저장 경로: c:\project-AI2\실프로젝트\data\processed\csv\case.csv


In [13]:
# 파일별 통계
if all_dfs:
    print("\n📈 파일별 통계:")
    stats = final_df.groupby('src_title').size()
    for title, count in stats.items():
        print(f"   - {title}: {count}개 청크")


📈 파일별 통계:
   - 상담사례 - 2025전세피해지원사례집: 50개 청크
   - 상담사례 - 전세피해법률상담사례집_250102: 110개 청크


In [14]:
# 우선순위별 통계
if all_dfs:
    print("\n📊 우선순위별 통계:")
    priority_stats = final_df.groupby('priority').size()
    priority_names = {5: "대법원판례", 6: "하급심판례", 7: "상담사례"}
    for priority, count in priority_stats.items():
        print(f"   - {priority_names.get(priority, f'Priority {priority}')}: {count}개")


📊 우선순위별 통계:
   - 상담사례: 160개


In [15]:
# 카테고리별 통계
if all_dfs:
    print("\n📊 카테고리별 통계:")
    all_categories = []
    for cats in final_df['category']:
        all_categories.extend([c.strip() for c in cats.split(',')])
    
    category_counts = Counter(all_categories)
    for cat, count in category_counts.most_common():
        print(f"   - {cat}: {count}개")


📊 카테고리별 통계:
   - 보증금_대항력: 145개
   - 권리_리스크: 130개
   - 분쟁해결: 104개
   - 임대료_증감: 26개
   - 임차권_승계: 19개
   - 계약해지: 17개
   - 계약갱신: 6개
   - 수선_원상회복: 3개
   - 행정절차: 2개
   - 기타: 2개


### 8. 결과 미리보기

In [16]:
if all_dfs:
    print("\n📋 CSV 미리보기 (첫 5개):")
    display(final_df.head())


📋 CSV 미리보기 (첫 5개):


,id,chunk_id,text,src_title,case_no,ruling_date,related_article,category,priority,source,parsed_at
0,11b18b5c-0e7b-4ba4-8662-336ad7054ad7,chunk_000,전세피해지원 프로그램 및\n전세피해 상담 사례집\n 국토부 전세사기피해자 및 HU...,상담사례 - 2025전세피해지원사례집,미상,미상,주임법 제8조,"분쟁해결, 보증금_대항력, 권리_리스크",7,c:\project-AI2\실프로젝트\data\raw\case\2025전세피해지원사...,2026-01-19T17:30:57.170122
1,ab27340e-a006-4c30-ae17-ba4361fcbdfd,chunk_001,8. 전세사기피해자 결정 이전 긴급 경·공매 유예·정지 32 \n아직 국토부 전...,상담사례 - 2025전세피해지원사례집,미상,미상,주임법 제8조,"권리_리스크, 분쟁해결, 보증금_대항력",7,c:\project-AI2\실프로젝트\data\raw\case\2025전세피해지원사...,2026-01-19T17:30:57.171122
2,bbf7fff7-3914-477e-9e00-001ee97c3b3c,chunk_002,16. 경·공매 셀프낙찰로 인한 전세대출 연체 38 \n피해주택을 직접 낙찰 받...,상담사례 - 2025전세피해지원사례집,미상,미상,주임법 제8조,"권리_리스크, 보증금_대항력, 임차권_승계",7,c:\project-AI2\실프로젝트\data\raw\case\2025전세피해지원사...,2026-01-19T17:30:57.171122
3,b2a88410-4ed9-49b2-b415-12014ca8093f,chunk_003,l ‘전세사기피해자 지원 및 주거안정을 위한 특별법 (이하 전세사기 특별법) ’...,상담사례 - 2025전세피해지원사례집,미상,미상,주임법 제8조,"보증금_대항력, 권리_리스크, 분쟁해결",7,c:\project-AI2\실프로젝트\data\raw\case\2025전세피해지원사...,2026-01-19T17:30:57.171122
4,48665737-a0fa-4d00-87dd-7e3b32841b89,chunk_004,"보증 가입임차인이 주택임차보증금 반환 보증보험에 가입했거나, 임대인이 임대보증금 반...",상담사례 - 2025전세피해지원사례집,미상,미상,주임법 제8조,"보증금_대항력, 권리_리스크, 임대료_증감",7,c:\project-AI2\실프로젝트\data\raw\case\2025전세피해지원사...,2026-01-19T17:30:57.171122


In [17]:
# 특정 청크 확인
if all_dfs:
    print("\n🔍 청크 샘플:")
    sample = final_df.iloc[0]
    print(f"사건번호: {sample['case_no']}")
    print(f"선고일: {sample['ruling_date']}")
    print(f"참조조항: {sample['related_article']}")
    print(f"카테고리: {sample['category']}")
    print(f"우선순위: {sample['priority']}")
    print(f"본문(앞 200자): {sample['text'][:200]}...")


🔍 청크 샘플:
사건번호: 미상
선고일: 미상
참조조항: 주임법 제8조
카테고리: 분쟁해결, 보증금_대항력, 권리_리스크
우선순위: 7
본문(앞 200자): 전세피해지원 프로그램 및
전세피해 상담 사례집
  국토부 전세사기피해자 및 HUG 전세피해확인서 
1. 국토부 전세사기피해자 선정기준 05
2. HUG 전세피해확인서 발급기준 07
  전세피해지원 프로그램 
1. 전세피해 지원프로그램 요약  10
2. 전세피해 지원프로그램 현황  12
  주요 피해지원 상담 사례 
1.   전세대출 연체 문제  27 
전세...


### 9. 컬럼 정보 확인

In [ ]:
if all_dfs:
    print("\n📋 데이터프레임 정보:")
    print(final_df.info())
    print("\n컬럼 목록:")
    print(final_df.columns.tolist())


📋 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               160 non-null    object
 1   chunk_id         160 non-null    object
 2   text             160 non-null    object
 3   src_title        160 non-null    object
 4   case_no          160 non-null    object
 5   ruling_date      160 non-null    object
 6   related_article  160 non-null    object
 7   category         160 non-null    object
 8   priority         160 non-null    int64 
 9   source           160 non-null    object
 10  parsed_at        160 non-null    object
dtypes: int64(1), object(10)
memory usage: 13.9+ KB
None

컬럼 목록:
['id', 'chunk_id', 'text', 'src_title', 'case_no', 'ruling_date', 'related_article', 'category', 'priority', 'source', 'parsed_at']


: 